In [1]:
import numpy as np

np.random.seed(0)

# ⚙️ THÔNG SỐ
N = 8         # số subcarriers
symbols = 2   # 1 cột pilot, 1 cột data

# 🎯 TẠO LƯỚI TÀI NGUYÊN
resource_grid = np.zeros((N, symbols), dtype=complex)

# 🔹 CỘT 0: PILOT (BPSK ±1)
pilot = 2 * (np.random.randint(0, 2, N) - 0.5)  # ±1
resource_grid[:, 0] = pilot

# 🔹 CỘT 1: DỮ LIỆU (QPSK)
data_bits = np.random.randint(0, 2, N * 2).reshape(N, 2)
qpsk_mod = lambda b: ((1 if b[0] else -1) + 1j * (1 if b[1] else -1)) / np.sqrt(2)
data = np.array([qpsk_mod(b) for b in data_bits])
resource_grid[:, 1] = data

print("🔸 Resource Grid (Tần số):")
print(np.round(resource_grid, 3))

# 🎯 DFT-PRECODING cho cột data
data_precoded = np.fft.fft(resource_grid[:, 1])
resource_grid[:, 1] = data_precoded

# 🌀 IFFT từng cột → tín hiệu miền thời gian
tx_signal_time = np.fft.ifft(resource_grid, axis=0)
print("\n🔹 OFDM Time-Domain Signal:")
print(np.round(tx_signal_time, 3))

# ✅ TRUYỀN QUA KÊNH LÝ TƯỞNG (không fading, không nhiễu)
rx_signal_time = tx_signal_time.copy()

# 🔁 FFT tại đầu thu
rx_freq = np.fft.fft(rx_signal_time, axis=0)

# 🔄 BỎ DFT bằng IDFT với cột data (cột 1)
rx_data_no_precoding = np.fft.ifft(rx_freq[:, 1])

# 🧪 So sánh với QAM gốc
print("\n✅ So sánh đầu vào QPSK và đầu ra sau DFT-precoding:")
print("TX QPSK:", np.round(data, 3))
print("RX QPSK:", np.round(rx_data_no_precoding, 3))
print("Sai số tuyệt đối:", np.round(np.abs(data - rx_data_no_precoding), 4))


🔸 Resource Grid (Tần số):
[[-1.   +0.j     0.707+0.707j]
 [ 1.   +0.j     0.707-0.707j]
 [ 1.   +0.j    -0.707+0.707j]
 [-1.   +0.j    -0.707-0.707j]
 [ 1.   +0.j    -0.707-0.707j]
 [ 1.   +0.j    -0.707+0.707j]
 [ 1.   +0.j    -0.707+0.707j]
 [ 1.   +0.j     0.707-0.707j]]

🔹 OFDM Time-Domain Signal:
[[ 0.5  +0.j     0.707+0.707j]
 [-0.073-0.177j  0.707-0.707j]
 [-0.25 +0.25j  -0.707+0.707j]
 [-0.427-0.177j -0.707-0.707j]
 [ 0.   +0.j    -0.707-0.707j]
 [-0.427+0.177j -0.707+0.707j]
 [-0.25 -0.25j  -0.707+0.707j]
 [-0.073+0.177j  0.707-0.707j]]

✅ So sánh đầu vào QPSK và đầu ra sau DFT-precoding:
TX QPSK: [ 0.707+0.707j  0.707-0.707j -0.707+0.707j -0.707-0.707j -0.707-0.707j
 -0.707+0.707j -0.707+0.707j  0.707-0.707j]
RX QPSK: [ 0.707+0.707j  0.707-0.707j -0.707+0.707j -0.707-0.707j -0.707-0.707j
 -0.707+0.707j -0.707+0.707j  0.707-0.707j]
Sai số tuyệt đối: [0. 0. 0. 0. 0. 0. 0. 0.]
